In [20]:
from preprocessing.file_types.edf_preprocess import EDFFiles
import numpy as np
import pandas as pd
import re
import os
from tqdm import tqdm

In [33]:
def create_dataset(ecg_npy_array, seizure_start_seconds, seizure_end_seconds, idx):
    seizure_start_seconds = int(np.absolute(seizure_start_seconds))
    seizure_end_seconds = int(np.absolute(seizure_end_seconds))
    preictal = 0
    non_preictal = []
    # If seizure starts before 2400 seconds have elapsed, take everything upto when the seizure starts
    #For non-preictal data, if seizure starts within 2400 seconds, only take from seizure end: experiment end
    if seizure_start_seconds < 40*60: # 40 minutes * 60 seconds
        preictal = ecg_npy_array[:seizure_start_seconds*512]
        non_preictal.append(ecg_npy_array[seizure_end_seconds*512:len(ecg_npy_array)])
        
    # Else, if seizure starts after 2400 seconds, take seizure_start - 2400 upto seizure_start
    # For non-preictal data, take 0:seizure_start-2400 and seizure_end:experiment_end
    elif seizure_start_seconds > 40*60:
        preictal = ecg_npy_array[seizure_start_seconds*512-2400 : seizure_start_seconds*512]
        non_preictal.append(ecg_npy_array[:seizure_start_seconds*512-2400])
        non_preictal.append(ecg_npy_array[seizure_end_seconds*512:len(ecg_npy_array)])
        
    #Store data
    path_idx = 1
    save_path_preictal = os.path.join("D:/PacificAutism/data/dataset/preictal/", str(idx)+'.npy')
    # Store preictal data
    np.save(save_path_preictal, preictal)
    # Store non-preictal data
    for n_preictal in non_preictal:
        save_path_non_preictal = os.path.join("D:/PacificAutism/data/dataset/non_preictal/", str(idx)+'_'+str(path_idx)+'.npy')
        np.save(save_path_non_preictal, n_preictal)
        path_idx += 1

In [34]:
ecg_data_path = r'D:/PacificAutism/data/Patient_ECG_signals'
seizure_start_path = r'D:/PacificAutism/data/Patient_dict/seizure_start_seconds.pkl'
seizure_start = pd.read_pickle(seizure_start_path)
seizure_end_path = r'D:/PacificAutism/data/Patient_dict/seizure_end_seconds.pkl'
seizure_end = pd.read_pickle(seizure_end_path)

save_idx = 0
for patient_ecg_data_path in tqdm(os.listdir(ecg_data_path)):
    this_ecg_data = np.load(os.path.join(ecg_data_path, patient_ecg_data_path))
    if '_' in patient_ecg_data_path:
        patient_name = patient_ecg_data_path.split('_')[0]
        patient_idx = int(re.findall(r'\d+', patient_ecg_data_path.split('_')[1].split('.')[0])[0]) - 1
    else:
        patient_name = patient_ecg_data_path.split('.')[0]
        patient_idx = 0

    create_dataset(
        ecg_npy_array=this_ecg_data, 
        seizure_start_seconds=seizure_start[patient_name][patient_idx], 
        seizure_end_seconds=seizure_end[patient_name][patient_idx],
        idx=save_idx
    )
    save_idx += 1

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:30<00:00,  1.06s/it]


In [46]:
path = r"D:/PacificAutism/data/dataset/preictal/10.npy"

len(np.load(path))

2400